<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/FL_DP_PSO_SGD/FL_DP_PSO_SGD_Iris_gradient_clipping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pega o erro de cada um e compara. A partícula com menor erro pede o gbest.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import csv
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
#Hiperparâmetros
BATCH_SIZE = 64
EPOCHS_PER_CLIENT = 10
NUM_CLIENTS = 10
NUM_ROUNDS = 100
INERTIA, C1, C2 = 0.9, 0.8, 0.9
# LEARNING_RATE = 0.001
EPSILON = 5
DELTA = 1e-5
SENSITIVITY = 4
MAX_NORM = 2
# Configuração do dispositivo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

training on cuda


In [10]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)


# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

# # Função para salvar os resultados em um arquivo CSV
# def write_csv(algorithm_name, architecture, results):
#     file_name = f'{algorithm_name}_{architecture}_PreTreinado_CIFAR10_output.csv'
#     with open(file_name, 'w', encoding='utf-8', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(["Epoch", "Test Loss", "Test Accuracy"])
#         for i, result in enumerate(results):
#             writer.writerow([i + 1] + result)


class MLP(nn.Module): # Class MLP heritage from Module
  def __init__(self, input_dim, output_dim): # Constructor. 'self' in Python is similar to 'this' in C++.
    super(MLP, self).__init__() # Call Module constructor first to guarantee initialization
    # Defining each layer: type and number of neurons
    self.fc1 = nn.Linear(input_dim, 16)
    self.fc2 = nn.Linear(16, 8)
    self.fc3 = nn.Linear(8, 8)
    self.fc4 = nn.Linear(8, output_dim)

    # Defining each activation function
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x) # No activation function
    return x

class Particle:
    def __init__(self, particle_id, model, data):
        self.particle_id = particle_id
        self.model = model.to(DEVICE)
        self.data = data  # `self.data` agora armazena o dataset, não o DataLoader
        self.local_best_model = copy.deepcopy(model.state_dict())
        self.local_best_score = float('inf')
        self.velocity = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.007, weight_decay=1e-5)

    def train_particle_adam(self, criterion, EPSILON, DELTA, SENSITIVITY, MAX_NORM):
        self.model.train()
        train_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=True)
        for _ in range(EPOCHS_PER_CLIENT):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()

                sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON

                torch.nn.utils.clip_grad_norm_(self.model.parameters(), MAX_NORM)
                for param in self.model.parameters():
                    if param.grad is not None:
                        noise = torch.normal(mean=0, std=sigma, size=param.grad.shape, device=param.grad.device)
                        param.grad += noise

                self.optimizer.step()

        val_loss = self.evaluate_loss(criterion)
        if val_loss < self.local_best_score:
            self.local_best_score = val_loss
            self.local_best_model = copy.deepcopy(self.model.state_dict())
        return self.particle_id, val_loss

    def train_particle_pso(self, global_best_model, INERTIA, C1, C2):
        for name, param in self.model.named_parameters():
            if param.grad is None:
                continue
            local_rand = random.random()
            global_rand = random.random()
            self.velocity[name] = (
                INERTIA * self.velocity[name]
                + C1 * local_rand * (self.local_best_model[name].to(DEVICE) - param.data)
                + C2 * global_rand * (global_best_model[name].to(DEVICE) - param.data)
            )
            param.data += self.velocity[name]
            # param.data = param.data - LEARNING_RATE * param.grad
    def evaluate_loss(self, criterion):
        self.model.eval()
        total_loss = 0
        val_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=False)
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(val_loader)


def train_federated():
    # transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.ToTensor(),
    #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    # ])

    # trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    # trainloaders = [create_subset(trainset, 5000) for _ in range(NUM_CLIENTS)]

    # testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    # testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    iris = datasets.load_iris() # Load dataset Iris. Classification. 150 instances, 4 features and 3 classes.
    X = iris.data # X receives all features of each flower
    y = iris.target # y receives all classes of each flower

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Spliting data: 20% to test and 80% to training

    # Normalizing data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Transforming data in tensor
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.long)

        # Criando datasets
    trainset = TensorDataset(X_train, y_train)
    testset = TensorDataset(X_test, y_test)

    # Criando subconjuntos para cada cliente
    SUBSET_SIZE = 12
    trainloaders = [create_subset(trainset, SUBSET_SIZE) for _ in range(NUM_CLIENTS)]

    # Criando DataLoader para o conjunto de teste
    testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    input_dim = X_train.shape[1]  # Número de atributos (4 para o Iris)
    output_dim = len(torch.unique(y_train))  # Número de classes (3 para o Iris)

    server_model = MLP(input_dim, output_dim).to(DEVICE)
    particles = [Particle(i, copy.deepcopy(server_model), trainloaders[i]) for i in range(NUM_CLIENTS)]
    criterion = nn.CrossEntropyLoss()

    global_best_score = float('inf')
    global_best_model = copy.deepcopy(server_model.state_dict())
    server_evaluate_acc = []

    for epoch in range(NUM_ROUNDS):
        server_result = []
        for particle in particles:
            if epoch > 0:
                particle.train_particle_pso(global_best_model, INERTIA, C1, C2)
            pid, val_loss = particle.train_particle_adam(criterion, EPSILON, DELTA, SENSITIVITY, MAX_NORM)
            server_result.append((pid, val_loss))

        # print(server_result)
        sorted_result = sorted(server_result, key=lambda x: x[1])
        # print(sorted_result)
        top_3_particles = sorted_result[:3]
        # print(top_3_particles)

        # best_particle = min(server_result, key=lambda x: x[1])
        best_particle = random.choice(top_3_particles)
        best_particle_id = best_particle[0]  # ID da melhor partícula
        global_best_score = best_particle[1]
        global_best_model = copy.deepcopy(particles[best_particle_id].local_best_model)

        if (epoch + 1) % 10 == 0:
          print(f'Epoch {epoch + 1}/{NUM_ROUNDS}, Best Particle Loss: {global_best_score:.4f}')
          print(f"Round {epoch + 1}: Partícula {best_particle_id} está enviando os pesos.")

        server_model.load_state_dict(global_best_model)
        server_model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = server_model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

        test_loss = total_loss / len(testloader)
        test_accuracy = (correct / len(testset))*100
        server_evaluate_acc.append([test_loss, test_accuracy])
        if (epoch + 1) % 10 == 0:
          print(f'Epoch {epoch + 1}/{NUM_ROUNDS}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
          print('---')

    #write_csv("FLPSO-SGD3", architecture, server_evaluate_acc)

# Exemplo de execução
train_federated()

Epoch 10/100, Best Particle Loss: 1.1132
Round 10: Partícula 3 está enviando os pesos.
Epoch 10/100, Test Loss: 1.1615, Test Accuracy: 30.00%
---
Epoch 20/100, Best Particle Loss: 0.9220
Round 20: Partícula 0 está enviando os pesos.
Epoch 20/100, Test Loss: 1.0036, Test Accuracy: 33.33%
---
Epoch 30/100, Best Particle Loss: 0.3337
Round 30: Partícula 8 está enviando os pesos.
Epoch 30/100, Test Loss: 0.5968, Test Accuracy: 70.00%
---
Epoch 40/100, Best Particle Loss: 0.2709
Round 40: Partícula 8 está enviando os pesos.
Epoch 40/100, Test Loss: 0.4596, Test Accuracy: 73.33%
---
Epoch 50/100, Best Particle Loss: 0.1925
Round 50: Partícula 8 está enviando os pesos.
Epoch 50/100, Test Loss: 0.5223, Test Accuracy: 70.00%
---
Epoch 60/100, Best Particle Loss: 0.1043
Round 60: Partícula 5 está enviando os pesos.
Epoch 60/100, Test Loss: 0.1507, Test Accuracy: 96.67%
---
Epoch 70/100, Best Particle Loss: 0.0667
Round 70: Partícula 2 está enviando os pesos.
Epoch 70/100, Test Loss: 0.5810, Test